In [3]:
import torch
import torchvision.transforms as TF
from cl_lite.head.dynamic_simple import DynamicSimpleHead
import cl_lite.backbone as B
import os
import numpy as np
import sys
sys.path.append('..')
from rdfcil.datamodule import DataModule
# import random
# random.seed(1)
# dataset = "imagenet100"
dataset = "imagenet1000"
num_classes = 1000
data_root = '../rdfcil/data'
class_order = [542, 618, 816, 68, 94, 215, 60, 585, 942, 165, 968, 829, 186, 661, 525, 982, 315, 52, 641, 797, 49, 40, 731, 260, 467, 563, 251, 595, 299, 187, 742, 997, 531, 802, 459, 349, 695, 159, 334, 50, 546, 196, 873, 181, 795, 433, 807, 691, 53, 624, 820, 440, 338, 327, 562, 374, 311, 197, 20, 377, 786, 184, 512, 859, 406, 271, 911, 629, 951, 482, 567, 633, 720, 728, 250, 84, 583, 396, 418, 297, 534, 245, 480, 170, 910, 600, 497, 382, 891, 67, 767, 732, 161, 234, 185, 775, 825, 926, 853, 31, 83, 664, 471, 642, 137, 26, 163, 778, 885, 553, 265, 979, 511, 640, 93, 429, 560, 946, 307, 99, 759, 470, 967, 934, 632, 204, 505, 180, 827, 208, 707, 354, 856, 524, 362, 703, 360, 188, 160, 782, 739, 613, 42, 384, 287, 8, 107, 724, 272, 989, 372, 915, 488, 704, 472, 597, 930, 124, 804, 685, 486, 516, 593, 499, 537, 25, 495, 594, 87, 834, 945, 233, 123, 296, 367, 509, 626, 588, 922, 183, 608, 996, 647, 48, 73, 446, 713, 178, 686, 63, 15, 77, 154, 734, 116, 858, 957, 41, 368, 774, 326, 273, 176, 961, 727, 190, 701, 714, 373, 314, 964, 148, 376, 447, 58, 275, 331, 596, 735, 678, 398, 88, 921, 65, 697, 601, 308, 491, 956, 276, 415, 468, 259, 150, 699, 940, 605, 421, 388, 669, 681, 508, 999, 210, 954, 655, 202, 281, 416, 175, 513, 984, 651, 606, 990, 448, 478, 635, 345, 100, 680, 819, 584, 46, 826, 56, 763, 340, 199, 779, 937, 682, 895, 616, 994, 980, 235, 361, 294, 749, 128, 501, 206, 938, 865, 902, 214, 718, 408, 652, 550, 420, 192, 168, 162, 350, 925, 203, 291, 404, 656, 410, 850, 357, 369, 551, 993, 748, 227, 610, 270, 108, 667, 47, 38, 860, 302, 267, 766, 896, 81, 962, 783, 705, 473, 221, 289, 86, 887, 280, 138, 844, 919, 676, 445, 888, 892, 544, 126, 385, 914, 106, 886, 901, 890, 932, 654, 322, 975, 174, 949, 815, 255, 658, 263, 874, 521, 504, 590, 708, 130, 522, 432, 793, 506, 897, 136, 266, 722, 153, 110, 257, 279, 133, 494, 884, 995, 125, 9, 514, 987, 242, 905, 213, 604, 799, 498, 434, 444, 425, 292, 758, 3, 688, 698, 674, 955, 442, 332, 988, 719, 428, 33, 226, 619, 157, 475, 225, 587, 439, 936, 29, 589, 706, 784, 212, 228, 117, 541, 342, 18, 417, 899, 864, 36, 625, 952, 861, 876, 931, 456, 752, 231, 325, 310, 532, 507, 881, 452, 684, 792, 243, 636, 769, 378, 262, 353, 492, 518, 768, 883, 4, 387, 424, 576, 109, 11, 438, 274, 223, 323, 781, 437, 236, 436, 12, 841, 131, 904, 570, 900, 304, 283, 716, 484, 122, 405, 913, 538, 603, 515, 127, 290, 230, 976, 730, 500, 998, 72, 401, 149, 764, 696, 831, 209, 523, 402, 811, 303, 787, 882, 319, 101, 578, 862, 463, 278, 663, 671, 738, 694, 79, 346, 66, 843, 268, 743, 709, 813, 810, 95, 264, 371, 637, 142, 381, 666, 380, 460, 750, 866, 286, 43, 469, 796, 772, 76, 28, 169, 917, 198, 172, 644, 485, 23, 848, 868, 923, 609, 269, 399, 75, 147, 679, 614, 628, 631, 950, 715, 496, 261, 867, 355, 972, 347, 833, 565, 98, 144, 412, 611, 453, 324, 561, 0, 575, 21, 702, 450, 528, 569, 313, 677, 218, 845, 790, 981, 851, 983, 668, 237, 740, 692, 571, 392, 113, 118, 824, 298, 765, 359, 54, 312, 295, 309, 805, 409, 135, 44, 339, 10, 670, 564, 344, 991, 193, 476, 241, 577, 726, 548, 906, 375, 683, 660, 238, 300, 155, 30, 545, 543, 536, 828, 140, 189, 78, 246, 59, 817, 580, 789, 427, 907, 305, 182, 971, 591, 336, 120, 893, 760, 449, 207, 530, 114, 403, 510, 27, 479, 285, 35, 773, 808, 370, 944, 166, 17, 337, 672, 321, 840, 112, 343, 533, 943, 320, 24, 721, 977, 935, 855, 164, 253, 430, 158, 800, 348, 880, 659, 687, 220, 413, 916, 798, 953, 847, 82, 389, 547, 39, 785, 474, 34, 141, 258, 306, 351, 121, 711, 85, 872, 244, 51, 777, 328, 517, 145, 352, 477, 89, 737, 91, 483, 751, 650, 970, 419, 191, 216, 167, 391, 379, 832, 143, 329, 646, 963, 487, 139, 689, 195, 249, 909, 854, 986, 929, 535, 173, 37, 941, 211, 657, 889, 958, 57, 451, 973, 5, 948, 823, 645, 729, 16, 356, 366, 812, 426, 301, 552, 803, 390, 397, 928, 22, 947, 443, 414, 836, 288, 490, 622, 177, 200, 252, 771, 555, 458, 755, 835, 365, 529, 620, 69, 6, 599, 341, 627, 394, 878, 852, 673, 842, 746, 582, 717, 364, 879, 700, 649, 745, 791, 648, 592, 780, 333, 316, 71, 152, 756, 801, 32, 849, 723, 744, 690, 924, 1, 912, 978, 573, 838, 201, 725, 282, 134, 317, 693, 821, 927, 407, 386, 489, 788, 857, 757, 818, 830, 959, 217, 247, 132, 540, 733, 869, 229, 171, 493, 457, 119, 194, 623, 103, 422, 464, 503, 411, 102, 502, 607, 639, 920, 92, 568, 558, 74, 933, 814, 960, 908, 615, 62, 903, 992, 383, 839, 363, 179, 974, 612, 423, 239, 939, 837, 248, 965, 205, 586, 64, 762, 871, 256, 526, 70, 461, 454, 634, 393, 794, 520, 156, 966, 675, 598, 462, 13, 90, 875, 55, 45, 643, 630, 753, 572, 19, 61, 104, 581, 602, 754, 665, 105, 435, 662, 559, 14, 863, 539, 519, 761, 232, 554, 441, 574, 747, 653, 877, 224, 293, 556, 358, 898, 806, 712, 254, 97, 710, 736, 621, 146, 465, 431, 330, 395, 129, 741, 579, 115, 111, 549, 80, 222, 617, 969, 809, 527, 776, 151, 822, 870, 240, 277, 7, 918, 466, 400, 557, 566, 846, 985, 481, 219, 318, 894, 2, 770, 335, 284, 638, 96, 455]
# class_order = [68, 56, 78, 8, 23, 84, 90, 65, 74, 76, 40, 89, 3, 92, 55, 9, 26, 80, 43, 38, 58, 70, 77, 1, 85, 19, 17, 50, 28, 53, 13, 81, 45, 82, 6, 59, 83, 16, 15, 44, 91, 41, 72, 60, 79, 52, 20, 10, 31, 54, 37, 95, 14, 71, 96, 98, 97, 2, 64, 66, 42, 22, 35, 86, 24, 34, 87, 21, 99, 0, 88, 27, 18, 94, 11, 12, 47, 25, 30, 46, 62, 69, 36, 61, 7, 63, 75, 5, 32, 4, 51, 48, 73, 93, 39, 67, 29, 49, 57, 33]
num_tasks = 10

# Convert class_order to a tensor for faster indexing
class_order_tensor = torch.tensor(class_order).cuda()

# Create a tensor of zeros with the same length as class_order
# This tensor will be used to create a mapping where the index is the class order position
mapping_tensor = torch.zeros(len(class_order), dtype=torch.long).cuda()

# Assign the new class indices (which are just the indices of class_order_tensor) to the corresponding positions in mapping_tensor
mapping_tensor[class_order_tensor] = torch.arange(len(class_order_tensor)).cuda()

# Use the mapping tensor to map the labels
# fast_mapped_labels = mapping_tensor[labels]

In [5]:
# get forgetting results
total_task_acc=[]
for t in range(1,num_tasks+1):
    print(f"Task {t}")
    
    # get the model
    if dataset.startswith("imagenet"):
        backbone = B.resnet.resnet18()
    else:
        backbone = ISCF_ResNet()

    # prefix = './ImageNet-100/imnet100_version_675_rdfcil_5task_49.44/task_{}'.format(t-1)
    # prefix = './ImageNet-100/version_508_imnet_5task_54.64/task_{}'.format(t-1)
    # prefix = './ImageNet-100/version_507_imnet_20task_32.90/task_{}/'.format(t-1)
    # prefix = './ImageNet-100/version_793_imnet_20t_rdfcil/task_{}/'.format(t-1)
    # prefix = './ImageNet-100/version_451_imnet_10task_45.18/task_{}/'.format(t-1)
    # prefix = './ImageNet-100/imnet100_version_430_rdfcil_10task_40.7/task_{}/'.format(t-1)
    prefix = './ImageNet-100/version_596_imnet1000_10t_rdfcil/task_{}'.format(t-1)
    # prefix = './ImageNet-100/version_695_rdfcil_imnet1000_5t_19.53/task_{}'.format(t-1)
    # prefix = './ImageNet-100/version_611_imagenet_1000_iscf/task_{}'.format(t-1)
    state_dict = torch.load(os.path.join(prefix,"checkpoints/best_acc.ckpt"))['state_dict']

    # dataload
    data_module = DataModule(root=data_root, 
                            dataset=dataset, 
                            batch_size=128, 
                            num_workers=4,
                            num_tasks=num_tasks,
                            class_order=class_order,
                            current_task=t-1,
                            )
    data_module.setup()
    head = DynamicSimpleHead(num_classes=data_module.num_classes, num_features=backbone.num_features, bias=False)
    # head = DynamicSimpleHead(num_classes=data_module.num_classes, num_features=backbone.num_features, bias=True)
    
    backbone_state= {}
    head_state = {}
    for _ in range(t-1):
        head.append(num_classes//num_tasks)
    for k,v in state_dict.items():
        if k.startswith('backbone'):
            backbone_state[k[9:]] = v
        elif k.startswith('head'):
            head_state[k[5:]] = v
            # head_state[k[17:]] = v
            
        
    backbone.load_state_dict(backbone_state)
    backbone.eval()
    head.load_state_dict(head_state)


    # train_dataloader = data_module.train_dataloader()
    val_dataloader = data_module.val_dataloader()

    backbone.cuda()
    head.cuda()

    task_correct= [0 for _ in range(t)]
    task_total = [0 for _ in range(t)]
    idx=0
    for batch in val_dataloader:
        images, labels = batch
        images = images.cuda()
        labels = labels.cuda()
        with torch.no_grad():
            output = backbone(images)
            output = head(output)
            # print(output[0],labels[0])
            labels = mapping_tensor[labels]
            for i in range(t):
                t_indices = torch.nonzero(torch.bitwise_and(num_classes//num_tasks*(i+1) >= labels, num_classes//num_tasks*(i) < labels) ).view(-1)
                # print(t_indices.view(-1))
                # task accuracy
                labels_t = labels[t_indices] # - i*num_classes//num_tasks
                output_t = output[t_indices]
                # if i==0:
                #     output_t = output_t[:,:num_classes//num_tasks*(i+1)]
                # else: output_t = output_t[:,num_classes//num_tasks*i:num_classes//num_tasks*(i+1)]
                task_correct[i] += (output_t.argmax(dim=1) == labels_t).sum().item()
                task_total[i] += len(labels_t)
        idx+=1
        print('\r idx: {}'.format(idx), end='')
    print()
    task_acc = [float(cc)/ct for cc,ct in zip(task_correct,task_total)]
    print(task_acc)

    for j in range(num_tasks-t):
        task_acc.append(0)
    total_task_acc.append(task_acc)
total_task_acc = np.array(total_task_acc)
print(total_task_acc)
result = []
for i in range(num_tasks):
    if i == 0:
        result.append(0)
    else:
        res = 0
        for j in range(i + 1):
            res += (np.max(total_task_acc[:, j]) - total_task_acc[i][j])
        res = res / i
        result.append(100 * res)

result.pop(0)
print('Forgetting result:')
print(result)
print(sum(result)/len(result))

Task 1
[542, 618, 816, 68, 94, 215, 60, 585, 942, 165, 968, 829, 186, 661, 525, 982, 315, 52, 641, 797, 49, 40, 731, 260, 467, 563, 251, 595, 299, 187, 742, 997, 531, 802, 459, 349, 695, 159, 334, 50, 546, 196, 873, 181, 795, 433, 807, 691, 53, 624, 820, 440, 338, 327, 562, 374, 311, 197, 20, 377, 786, 184, 512, 859, 406, 271, 911, 629, 951, 482, 567, 633, 720, 728, 250, 84, 583, 396, 418, 297, 534, 245, 480, 170, 910, 600, 497, 382, 891, 67, 767, 732, 161, 234, 185, 775, 825, 926, 853, 31, 83, 664, 471, 642, 137, 26, 163, 778, 885, 553, 265, 979, 511, 640, 93, 429, 560, 946, 307, 99, 759, 470, 967, 934, 632, 204, 505, 180, 827, 208, 707, 354, 856, 524, 362, 703, 360, 188, 160, 782, 739, 613, 42, 384, 287, 8, 107, 724, 272, 989, 372, 915, 488, 704, 472, 597, 930, 124, 804, 685, 486, 516, 593, 499, 537, 25, 495, 594, 87, 834, 945, 233, 123, 296, 367, 509, 626, 588, 922, 183, 608, 996, 647, 48, 73, 446, 713, 178, 686, 63, 15, 77, 154, 734, 116, 858, 957, 41, 368, 774, 326, 273, 176, 961,

In [ ]:

forgetting = np.max(total_task_acc,axis=0) - np.min(total_task_acc,axis=0)
print(forgetting)
avg_forgetting = np.mean(forgetting)
print(avg_forgetting)